In [17]:
from sdf import gen_clientid, gen_managerid
import pandas as pd
from random import choice
from faker import Faker
from datetime import datetime, timedelta
from numpy import percentile

fake = Faker()

# Генерация данных

In [18]:
with open("titles.txt", encoding="utf-8") as file:
    titles = file.readlines()

titles = [x.strip() for x in titles]

In [19]:
tasks = pd.DataFrame(columns=['client_id', 'created_datetime', 'title'])

from_ = datetime(2018, 1, 1, 00, 00)
to_ = datetime(2022, 12, 31, 23, 59)

tasks_len = 25_000

tasks['client_id'] = [gen_clientid() for _ in range(tasks_len)]
tasks['created_datetime'] = [fake.date_time_between(start_date=from_, end_date=to_) for _ in range(tasks_len)]
tasks['title'] = [choice(titles) for _ in range(tasks_len)]

print(tasks.client_id.nunique(), tasks.created_datetime.nunique(), sep='\n')

24676
24998


In [20]:
created_datetime_first = tasks.groupby('client_id')['created_datetime'].min()

In [21]:
calls = pd.DataFrame(columns=['manager_id', 'client_id', 'call_datetime'])

calls_len = 35000

calls['manager_id'] = [gen_managerid() for _ in range(calls_len)]
calls['client_id'] = [choice(tasks.client_id) for _ in range(calls_len)]
call_datetime_ = []
for i in range(calls_len):
    from_ = created_datetime_first.loc[calls['client_id'][i]]
    to_ = (from_ + timedelta(weeks=1)) if (from_ + timedelta(weeks=1)).year <= 2022 else datetime(2022, 12, 31, 23, 59)
    call_datetime_.append(fake.date_time_between(start_date=from_, end_date=to_))
calls['call_datetime'] = call_datetime_

print(calls.manager_id.nunique(), calls.call_datetime.nunique(), sep='\n')

1000
34992


In [22]:
df = pd.merge(tasks, calls, on='client_id', how='left')
df['manager_id'] = df['manager_id'].astype('Int64')
df

,client_id,created_datetime,title,manager_id,call_datetime
0,1838916,2019-04-23 20:09:48,3.3 Как сделать перенаправление?,2523,2019-04-24 21:05:48
1,1838916,2019-04-23 20:09:48,3.3 Как сделать перенаправление?,2854,2019-04-23 21:52:42
2,1838916,2019-04-23 20:09:48,3.3 Как сделать перенаправление?,2348,2019-04-27 12:12:19
3,1838916,2019-04-23 20:09:48,3.3 Как сделать перенаправление?,2646,2019-04-27 10:32:29
4,1770588,2022-07-28 21:05:28,1.2 Как понять язык википедистов?,<NA>,NaT
...,...,...,...,...,...
41845,1649265,2018-02-03 12:42:23,2.1 Забыл пароль,2239,2018-02-09 09:24:24
41846,1059500,2022-09-13 17:57:20,1.3 А здесь есть модераторы?,2468,2022-09-17 16:32:53
41847,1865205,2020-12-07 21:11:17,1.4 Можно скачать всю Википедию?,2314,2020-12-14 01:00:03
41848,1865205,2020-12-07 21:11:17,1.4 Можно скачать всю Википедию?,2135,2020-12-14 18:36:48


In [23]:
# tasks.to_excel('tasks.xlsx')
# tasks.to_csv('tasks.csv')
# calls.to_excel('calls.xlsx')
# calls.to_csv('calls.csv')
# df.to_excel('df.xlsx')

# Задание 2. Изучите данные и выполните следующие задачи.

### Сколько заявок приходило каждый день в июне 2022 года.

In [24]:
tasks_2022_06 = tasks.loc[(tasks.created_datetime.dt.year == 2022) &
                          (tasks.created_datetime.dt.month == 6)]
tasks_2022_06.groupby(tasks.created_datetime.dt.day)['created_datetime'].count()

created_datetime
1     15
2     10
3     17
4     13
5     14
6     19
7     16
8     14
9     10
10     8
11     6
12    13
13    12
14     9
15    18
16    13
17     8
18    13
19    13
20    14
21    11
22    16
23    13
24    17
25    17
26    14
27    16
28    25
29    18
30    12
Name: created_datetime, dtype: int64

### Список тем, для которых обращений было больше 10(*) в апреле 2022 года

_*_ -- 65-ти процентный перцентиль

In [25]:
tasks_2022_04 = tasks.loc[(tasks.created_datetime.dt.year == 2022) &
                          (tasks.created_datetime.dt.month == 4)]
temp = tasks_2022_04.groupby('title')['created_datetime'].nunique().reset_index()

temp.loc[temp.created_datetime >= int(percentile(temp.created_datetime, 65)), :]

,title,created_datetime
1,1.1 Как работать в Википедии?,20
2,1.2 Как понять язык википедистов?,20
4,1.4 Можно скачать всю Википедию?,22
5,2.0 Регистрация,31
8,2.3 Хочу удалить свою учётную запись,21
9,2.4 Насколько родственны проекты Викимедиа?,22
11,3.1 Как добавить рисунок?,22
15,3.5 Как вставлять тире и ударения?,21
19,3.9 Как изменить заглавную страницу?,20


### Список клиентов, которые оставляли заявку, но ни одного звонка от менеджера по ним не было.

In [26]:
df[df['manager_id'].isna()]['client_id'].unique()

array([1770588, 1707700, 1359964, ..., 1663045, 1479720, 1823466],
      dtype=int64)

In [27]:
df[df['manager_id'].isna()]['client_id'].nunique()

5898

### Список клиентов, последние заявки которых не обработаны. Считаем, что если менеджер позвонил клиенту после заявки, то он ее обработал. Если заявок до звонка было несколько, то при звонке обрабатываются все заявки сразу.

In [28]:
df[df['created_datetime'] > df['call_datetime']]['client_id'].unique()

array([1889234, 1274188, 1861699, 1656189, 1821695, 1867048, 1920561,
       1981130, 1954068, 1364921, 1981642, 1048687, 1709481, 1796321,
       1454212, 1761977, 1090047, 1758726, 1604957, 1857504, 1027239,
       1262533, 1084857, 1104376, 1131389, 1401785, 1396667, 1286879,
       1099617, 1822707, 1040912, 1543412, 1040203, 1181796, 1301779,
       1143322, 1692744, 1385408, 1146028, 1962717, 1009015, 1057085,
       1988795, 1649944, 1640957, 1837265, 1455401, 1809006, 1760252,
       1955568, 1970071, 1341660, 1300738, 1945906, 1671186, 1087808,
       1304983, 1895455, 1425865, 1291197, 1466761, 1492164, 1835608,
       1888719, 1509589, 1317195, 1160122, 1742740, 1289567, 1800290,
       1530332, 1153717, 1460802, 1527247, 1047626, 1543794, 1572760,
       1928993, 1486903, 1117281, 1758109, 1142400, 1069841, 1222721,
       1824129, 1067960, 1807208, 1574195, 1422619, 1044456, 1623427,
       1535570, 1155178, 1932858, 1721189, 1434497, 1230734, 1966671,
       1739613, 1348

In [29]:
df[df['created_datetime'] > df['call_datetime']]['client_id'].nunique()

298

### Минимальную разницу между обращениями для каждого клиента, а затем вывести среднее по полученным значениям.

In [30]:
temp = df.dropna().copy()
temp['diff_datetime'] = (temp.call_datetime - temp.created_datetime).dt.days
temp = temp.loc[temp.diff_datetime > 0]

In [31]:
temp.groupby('client_id')['diff_datetime'].min()

client_id
1000015    6
1000094    5
1000162    1
1000395    5
1000406    6
          ..
1999538    2
1999717    6
1999802    5
1999924    2
1999927    2
Name: diff_datetime, Length: 17412, dtype: int64

In [32]:
temp.groupby('client_id')['diff_datetime'].min().mean().round(2)

2.92